In [2]:
#we finetune the inception v3 model by only changing the last layer
#and freezing all the other layers
import torch
import torch.nn as nn
import torchvision
import torchvision.models as models
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.optim as optim

In [3]:
# 1. Load the Inception model (visual representation below)
#The inception_v3 model in torchvision is pretrained on the ImageNet dataset.
#ImageNet is a large-scale dataset for image classification, containing over a million labeled images across 1000 classes.

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
d = torch.cuda.is_available()
if d==True:
    print("Torch on GPU available")
    print(f"GPU {torch.cuda.get_device_name(0)}")

model = models.inception_v3(pretrained=True).to(device)


Torch on GPU available
GPU Tesla T4


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:01<00:00, 103MB/s]


In [4]:
# [Image of Inception v3 architecture]

# 2. Freeze the pre-trained layers (preventing their weights from being updated)
for param in model.parameters():
    param.requires_grad = False
model.aux_logits = False

In [5]:
# 3. Replace the final fully connected layer with a new one for your task
# Modify the final fully connected layer for CIFAR-100 (100 classes)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 100,device=device)  # Change the output size to 100 classes


In [6]:
# 4. Define data transformations (visual examples included)
transform = transforms.Compose([
    transforms.Resize(299),  # Resize to Inception's input size
    transforms.CenterCrop(299),  # Center crop for consistency
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize to ImageNet stats
])


In [7]:
# 5. Load your dataset
# Load CIFAR-100 dataset
train_dataset = torchvision.datasets.CIFAR100(root='/content/drive/MyDrive/Colab Notebooks/Transfer_Learning', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR100(root='/content/drive/MyDrive/Colab Notebooks/Transfer_Learning', train=False, download=True, transform=transform)

# Create data loaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
# 6. Define loss function and optimizer
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [9]:
#model.train()
# 7. Train the model
num_epochs = 1
for epoch in range(num_epochs):
    running_loss = 0.0
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.cuda(), labels.cuda()

        optimizer.zero_grad()
        outputs = model(inputs)
        #outputs,_ = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # Print training information
        if batch_idx % 100 == 0:
            print(f"Epoch {epoch}/{num_epochs}, Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item():.4f}")

    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}")

Epoch 0/1, Batch 0/782, Loss: 4.6089
Epoch 0/1, Batch 100/782, Loss: 4.5024
Epoch 0/1, Batch 200/782, Loss: 4.4082
Epoch 0/1, Batch 300/782, Loss: 4.3149
Epoch 0/1, Batch 400/782, Loss: 4.1723
Epoch 0/1, Batch 500/782, Loss: 4.1046
Epoch 0/1, Batch 600/782, Loss: 3.9452
Epoch 0/1, Batch 700/782, Loss: 3.9481
Epoch 1/1, Loss: 4.2291


In [10]:
#Evaluation of the model
with torch.no_grad():
    correct = 0
    total = 0
    test_loss = 0
    for batch_idx,(inputs, labels) in enumerate(test_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        loss = criterion(outputs, labels)  # Use the same criterion as during training
        test_loss += loss.item()
        if batch_idx % 100 == 0:
            print(f"Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item():.4f}")
    epoch_loss = running_loss / len(train_loader)


Batch 0/782, Loss: 3.8975
Batch 100/782, Loss: 3.7544


In [11]:
accuracy = correct / total
print(f"Accuracy on the test set: {accuracy:.4f}")
print(f'Test Accuracy: {100 * accuracy:.2f}%')
print(f'Test Loss: {test_loss}')

Accuracy on the test set: 0.2880
Test Accuracy: 28.80%
Test Loss: 603.6114099025726
